In [35]:
import pandas as pd
import math

In [36]:
RIDERS = pd.read_csv("../data/riders.csv")
RIDER_TEAMS = pd.read_csv("../data/rider_teams.csv")
MANAGERS = pd.read_csv("../data/managers.csv")
MANAGER_TEAMS = pd.read_csv("../data/manager_teams.csv")
MANAGER_CHEAPO_TEAMS = pd.read_csv("../data/manager_cheapo_teams.csv")
POINTS_SYSTEM = pd.read_csv("../data/points_system.csv")
RACES = pd.read_csv("../data/races.csv")
RESULTS_2023 = pd.read_csv("../data/results/results_2023_full.csv")
RESULTS_2024 = pd.read_csv("../data/results/results_2024_full.csv")
RESULTS_2025 = pd.read_csv("../data/results/results_2025_full.csv")
CHEAPO_BANS = pd.read_csv("../data/cheapo_bans.csv")

In [37]:
# Riders owned by
RIDERS_OWNED_BY = MANAGER_TEAMS.groupby(["RiderName"])["ManagerName"].apply(", ".join).reset_index()
RIDERS_OWNED_BY_CHEAPO = MANAGER_CHEAPO_TEAMS.groupby(["RiderName"])["ManagerName"].apply(", ".join).reset_index()

# Race categories
RACE_CATEGORIES = RACES[["RaceName_PCS", "RaceCategory"]]

# Relevant results
RESULTS_POINTS = RESULTS_2025[["RACE", "STAGE_ID", "STAGE_DATE", "STAGE_TYPE", "RANK", "RIDER"]]
RESULTS_POINTS = RESULTS_POINTS.rename(columns = {
    "RACE": "RaceName",
    "STAGE_ID": "StageID",
    "STAGE_DATE": "StageDate",
    "STAGE_TYPE": "StageType",
    "RANK": "Rank",
    "RIDER": "RiderName"
})
RESULTS_POINTS = RESULTS_POINTS.dropna(subset = ["StageID"])
RESULTS_POINTS = RESULTS_POINTS.loc[RESULTS_POINTS["StageType"] != "TTT"]

def convert_to_int(row):
    if type(row) == int:
        return row
    if row.isdigit():
        return int(row)
    else:
        return 0
    
def stage_or_gcresult(stage_id, race_category):
    if stage_id.startswith("stage"):
        return race_category.replace("Tour ", "Stage ")
    else:
        return race_category    

RESULTS_POINTS = pd.merge(RESULTS_POINTS, RACE_CATEGORIES, how = "inner", left_on = ["RaceName"], right_on = ["RaceName_PCS"])
RESULTS_POINTS["Rank"] = RESULTS_POINTS.apply(lambda row: convert_to_int(row["Rank"]), axis = 1)
RESULTS_POINTS["RaceCategory"] = RESULTS_POINTS.apply(lambda row: stage_or_gcresult(row["StageID"], row["RaceCategory"]), axis = 1)
RESULTS_POINTS = pd.merge(RESULTS_POINTS, POINTS_SYSTEM, how = "left", left_on = ["RaceCategory", "Rank"], right_on = ["RaceCategory", "RaceRank"])
RESULTS_POINTS = pd.merge(RESULTS_POINTS, RIDERS, how = "left", left_on = ["RiderName"], right_on = ["RiderName_PCS"])

# Rider points
RIDER_POINTS = RESULTS_POINTS.groupby(["RiderName_Zweeler"])["RacePoints"].sum().reset_index()

# Rider racedays
RIDER_RACEDAYS = RESULTS_POINTS.loc[RESULTS_POINTS["StageID"] != "gc"]
RIDER_RACEDAYS = RIDER_RACEDAYS.groupby(["RiderName_Zweeler"])["RiderName_Zweeler"].count().rename("Racedays").reset_index()

In [38]:
RIDER_OUTPUT = RIDERS

RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDER_RACEDAYS, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName_Zweeler"])
RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDER_POINTS, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName_Zweeler"])
RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDERS_OWNED_BY, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName"])
RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDERS_OWNED_BY_CHEAPO, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName"])
RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, CHEAPO_BANS, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName"])

def ppm(points, millions):
    if math.isnan(points):
        return round(0, 2)
    else:
        return round(points / millions, 1)

def float_to_int(row):
    if math.isnan(row):
        return 0
    else:
        return int(row)

def cheapo_eligible(price, ban):
    if price <= 2.5 and ban == "":
        return "Ja"
    else:
        return ""
    
RIDER_OUTPUT["PPM"] = RIDER_OUTPUT.apply(lambda row: ppm(row["RacePoints"], row["RiderPrice"]), axis = 1)
RIDER_OUTPUT.RiderName = RIDER_OUTPUT.RiderName.fillna("")
RIDER_OUTPUT["Cheapo"] = RIDER_OUTPUT.apply(lambda row: cheapo_eligible(row["RiderPrice"], row["RiderName"]), axis = 1)
RIDER_OUTPUT["RacePoints"] = RIDER_OUTPUT.apply(lambda row: float_to_int(row["RacePoints"]), axis = 1)
RIDER_OUTPUT["Racedays"] = RIDER_OUTPUT.apply(lambda row: float_to_int(row["Racedays"]), axis = 1)
RIDER_OUTPUT.RiderName_x = RIDER_OUTPUT.RiderName_x.fillna("")
RIDER_OUTPUT.RiderName_y = RIDER_OUTPUT.RiderName_y.fillna("")
RIDER_OUTPUT.ManagerName_x = RIDER_OUTPUT.ManagerName_x.fillna("")
RIDER_OUTPUT.ManagerName_y = RIDER_OUTPUT.ManagerName_y.fillna("")

RIDER_OUTPUT = RIDER_OUTPUT[["RiderName_Zweeler", "RiderTeam", "RiderPrice", "Racedays", "RacePoints", "PPM", "Cheapo", "ManagerName_x", "ManagerName_y"]]

RIDER_OUTPUT = RIDER_OUTPUT.rename(columns = {
    "RiderName_Zweeler": "Navn",
    "RiderTeam": "Hold",
    "RiderPrice": "Pris",
    "Racedays": "Løbsdage",
    "RacePoints": "Point",
    "PPM": "Profit",
    "Cheapo": "Cheapo",
    "ManagerName_x": "Managers",
    "ManagerName_y": "Cheapo_Managers"
})

RIDER_OUTPUT_MAIN = RIDER_OUTPUT
RIDER_OUTPUT_MAIN = RIDER_OUTPUT_MAIN.drop(["Cheapo_Managers"], axis = 1)

RIDER_OUTPUT_CHEAPO = RIDER_OUTPUT
RIDER_OUTPUT_CHEAPO = RIDER_OUTPUT_CHEAPO.drop(["Managers"], axis = 1)
RIDER_OUTPUT_CHEAPO = RIDER_OUTPUT_CHEAPO.loc[RIDER_OUTPUT_CHEAPO["Cheapo"] == "Ja"]

In [39]:
# Standings
TEAMS_WITH_POINTS = pd.merge(MANAGER_TEAMS, RIDER_OUTPUT, how = "left", left_on = "RiderName", right_on = "Navn")
STANDINGS = TEAMS_WITH_POINTS.groupby(["ManagerName"])[["Pris", "Løbsdage", "Point"]].sum().reset_index()
STANDINGS["Point per løbsdag"] = round(STANDINGS["Point"] / STANDINGS["Løbsdage"], 2)
STANDINGS = STANDINGS.sort_values(by = "Point", ascending = False)
STANDINGS.rename(columns = {"ManagerName": "Manager", "Pris": "Samlet pris"})

,Manager,Samlet pris,Løbsdage,Point,Point per løbsdag
1,Hustlersen,327.8,82,579,7.06
0,Chrelle,350.0,59,483,8.19
4,Kenk,349.9,64,454,7.09
8,Tommy,350.0,71,325,4.58
6,Matti,349.1,48,319,6.65
2,Jappo,350.0,56,305,5.45
7,Okholm,350.0,57,300,5.26
9,Visti,350.0,50,265,5.30
5,Knak,349.8,57,262,4.60
3,Jarma,350.0,33,142,4.30


In [40]:
# Races
RACE_OUTPUT = RACES
RACE_OUTPUT = RACE_OUTPUT.drop("RaceName_PCS", axis = 1)
RACE_OUTPUT = RACE_OUTPUT.rename(columns = {
    "RaceNumber": "LøbID",
    "RaceName": "Løb",
    "RaceCategory": "Løbskategori",
    "RaceStart": "Løbsstart",
    "RaceEnd": "Løbsslut",
    "RaceLink_PCS": "Løbslink"
})

In [41]:
# Race points
RACE_POINTS = pd.merge(RESULTS_POINTS, RACES, how = "left", left_on = "RaceName", right_on = "RaceName_PCS")
RACE_POINTS = RACE_POINTS[["RaceName_y", "RaceCategory_y", "RiderName_Zweeler", "RiderTeam", "RacePoints", "StageID"]]

RACE_POINTS = RACE_POINTS.rename(columns = {
    "RaceName_y": "Løb",
    "RaceCategory_y": "Løbskategori",
    "RiderName_Zweeler": "Navn",
    "RiderTeam": "Hold",
    "RacePoints": "Point",
    "StageID": "StageID"
})

def result(id, point):
    if math.isnan(point):
        return ""
    if int(point) == 0:
        return ""
    else:
        return id + ": " + str(int(point))

RACE_POINTS["Resultater"] = RACE_POINTS.apply(lambda row: result(row["StageID"], row["Point"]), axis = 1)

RACE_POINTS_RACEDAYS = RACE_POINTS.groupby(["Løb", "Navn", "Løbskategori"]).count().reset_index()
RACE_POINTS_RACEDAYS = RACE_POINTS_RACEDAYS[["Løb", "Navn", "Løbskategori", "Hold"]]
RACE_POINTS_RACEDAYS = RACE_POINTS_RACEDAYS.rename(columns = {"Hold": "Løbsdage"})
RACE_POINTS_POINTS = RACE_POINTS.groupby(["Løb", "Navn", "Løbskategori"])["Point"].sum().reset_index()
RACE_POINTS_RESULTS = RACE_POINTS.loc[RACE_POINTS["Resultater"] != ""]
RACE_POINTS_RESULTS = RACE_POINTS_RESULTS.groupby(["Løb", "Navn", "Løbskategori"])["Resultater"].apply(" / ".join).reset_index()

RACE_POINTS = pd.merge(RACE_POINTS_RACEDAYS, RACE_POINTS_POINTS, how = "left", left_on = ["Løb", "Navn", "Løbskategori"], right_on = ["Løb", "Navn", "Løbskategori"])
RACE_POINTS = pd.merge(RACE_POINTS, RACE_POINTS_RESULTS, how = "left", left_on = ["Løb", "Navn", "Løbskategori"], right_on = ["Løb", "Navn", "Løbskategori"])
RACE_POINTS.Resultater = RACE_POINTS.Resultater.fillna("")
RACE_POINTS["Point"] = RACE_POINTS["Point"].astype(int)

In [42]:
# Dagens rapport

DAGENS_RAPPORT = pd.merge(RESULTS_POINTS, RIDERS_OWNED_BY, how = "left", left_on = "RiderName_Zweeler", right_on = "RiderName")
DAGENS_RAPPORT = pd.merge(DAGENS_RAPPORT, RIDERS_OWNED_BY_CHEAPO, how = "left", left_on = "RiderName_Zweeler", right_on = "RiderName")
DAGENS_RAPPORT = pd.merge(DAGENS_RAPPORT, RACES, how = "left", left_on = "RaceName", right_on = "RaceName_PCS")

def get_date(stage_date, race_date):
    if stage_date == "":
        return race_date[0:10]
    else:
        return stage_date
    
DAGENS_RAPPORT.StageDate = DAGENS_RAPPORT.StageDate.fillna("")
DAGENS_RAPPORT["Dato"] = DAGENS_RAPPORT.apply(lambda row: get_date(row["StageDate"], row["RaceEnd"]), axis = 1)

DAGENS_RAPPORT = DAGENS_RAPPORT[["RiderName_Zweeler", "RaceName_y", "Dato", "RacePoints", "ManagerName_x", "ManagerName_y"]]

DAGENS_RAPPORT = DAGENS_RAPPORT.rename(columns = {
    "RiderName_Zweeler": "Rytter",
    "RaceName_y": "Løb",
    "RacePoints": "Point",
    "ManagerName_x": "Managers_Zweeler",
    "ManagerName_y": "Managers_Cheapo"
})
DAGENS_RAPPORT.Managers_Zweeler = DAGENS_RAPPORT.Managers_Zweeler.fillna("")
DAGENS_RAPPORT.Managers_Cheapo = DAGENS_RAPPORT.Managers_Cheapo.fillna("")
DAGENS_RAPPORT.Point = DAGENS_RAPPORT.Point.fillna(0)
DAGENS_RAPPORT = DAGENS_RAPPORT[(DAGENS_RAPPORT["Managers_Zweeler"] != "") | (DAGENS_RAPPORT["Managers_Cheapo"] != "")]
DAGENS_RAPPORT = DAGENS_RAPPORT[(DAGENS_RAPPORT["Point"] > 0)]
DAGENS_RAPPORT = DAGENS_RAPPORT.astype({"Point": int})
DAGENS_RAPPORT_DATOER = DAGENS_RAPPORT["Dato"].unique().tolist()
DAGENS_RAPPORT_DATOER.sort(reverse = True)
DAGENS_RAPPORT = DAGENS_RAPPORT.groupby(["Rytter", "Løb", "Dato", "Managers_Zweeler", "Managers_Cheapo"])["Point"].sum().reset_index()


In [43]:
FULL_TEXT = """---
title: Dagens rapport
hide:
  - toc
---

"""

for DATO in DAGENS_RAPPORT_DATOER:
    RELEVANT = DAGENS_RAPPORT[DAGENS_RAPPORT["Dato"] == DATO]
    RELEVANT = RELEVANT.sort_values(by = "Point", ascending = False)
    RELEVANT_HOVED = RELEVANT[RELEVANT["Managers_Zweeler"] != ""]
    RELEVANT_CHEAPO = RELEVANT[RELEVANT["Managers_Cheapo"] != ""]
    RELEVANT_CHEAPO = RELEVANT_CHEAPO[RELEVANT_CHEAPO["Dato"] >= "2025-01-27"]

    FULL_TEXT = FULL_TEXT + "## " + DATO + ""

    FULL_TEXT = FULL_TEXT + "\n" + "### Hovedspillet"

    for index, row in RELEVANT_HOVED.iterrows():
        FULL_TEXT = FULL_TEXT + "\n" + row["Rytter"] + " **" + str(row["Point"]) + "** (" + row["Managers_Zweeler"] + ")  "

    FULL_TEXT = FULL_TEXT + "\n\n" + "### Cheapo"

    for index, row in RELEVANT_CHEAPO.iterrows():
        FULL_TEXT = FULL_TEXT + "\n" + row["Rytter"] + " **" + str(row["Point"]) + "** (" + row["Managers_Cheapo"] + ")  "

    FULL_TEXT = FULL_TEXT + "\n\n\n\n\n\n"

with open("../DagensRapport.md", "w") as file:
    file.write(FULL_TEXT)

In [ ]:
RIDERS_CATEGORIES = RIDERS[["RiderName_Zweeler"]]

RACE_POINTS["Løbskategori"].astype("str")

RACE_POINTS_TOUR1 = RACE_POINTS.loc[RACE_POINTS["Løbskategori"] == "Tour Cat. 1"][["Navn", "Point"]].groupby(["Navn"]).sum().reset_index()
RACE_POINTS_TOUR2 = RACE_POINTS[RACE_POINTS["Løbskategori"] == "Tour Cat. 2"][["Navn", "Point"]].groupby(["Navn"]).sum().reset_index()
RACE_POINTS_TOUR3 = RACE_POINTS[RACE_POINTS["Løbskategori"] == "Tour Cat. 3"][["Navn", "Point"]].groupby(["Navn"]).sum().reset_index()
RACE_POINTS_CLASSICS1 = RACE_POINTS[RACE_POINTS["Løbskategori"] == "Classics Cat. 1"][["Navn", "Point"]].groupby(["Navn"]).sum().reset_index()
RACE_POINTS_CLASSICS2 = RACE_POINTS[RACE_POINTS["Løbskategori"] == "Classics Cat. 2"][["Navn", "Point"]].groupby(["Navn"]).sum().reset_index()
RACE_POINTS_CLASSICS3 = RACE_POINTS[RACE_POINTS["Løbskategori"] == "Classics Cat. 3"][["Navn", "Point"]].groupby(["Navn"]).sum().reset_index()

RIDERS_CATEGORIES = pd.merge(RIDERS_CATEGORIES, RACE_POINTS_TOUR1, how = "left", left_on = "RiderName_Zweeler", right_on = "Navn")
RIDERS_CATEGORIES = RIDERS_CATEGORIES[["RiderName_Zweeler", "Point"]]
RIDERS_CATEGORIES = RIDERS_CATEGORIES.rename(columns = {"Point": "Tour1"})

RIDERS_CATEGORIES = pd.merge(RIDERS_CATEGORIES, RACE_POINTS_TOUR2, how = "left", left_on = "RiderName_Zweeler", right_on = "Navn")
RIDERS_CATEGORIES = RIDERS_CATEGORIES[["RiderName_Zweeler", "Tour1", "Point"]]
RIDERS_CATEGORIES = RIDERS_CATEGORIES.rename(columns = {"Point": "Tour2"})

RIDERS_CATEGORIES = pd.merge(RIDERS_CATEGORIES, RACE_POINTS_TOUR3, how = "left", left_on = "RiderName_Zweeler", right_on = "Navn")
RIDERS_CATEGORIES = RIDERS_CATEGORIES[["RiderName_Zweeler", "Tour1", "Tour2", "Point"]]
RIDERS_CATEGORIES = RIDERS_CATEGORIES.rename(columns = {"Point": "Tour3"})

RIDERS_CATEGORIES = pd.merge(RIDERS_CATEGORIES, RACE_POINTS_CLASSICS1, how = "left", left_on = "RiderName_Zweeler", right_on = "Navn")
RIDERS_CATEGORIES = RIDERS_CATEGORIES[["RiderName_Zweeler", "Tour1", "Tour2", "Tour3", "Point"]]
RIDERS_CATEGORIES = RIDERS_CATEGORIES.rename(columns = {"Point": "Classics1"})

RIDERS_CATEGORIES = pd.merge(RIDERS_CATEGORIES, RACE_POINTS_CLASSICS2, how = "left", left_on = "RiderName_Zweeler", right_on = "Navn")
RIDERS_CATEGORIES = RIDERS_CATEGORIES[["RiderName_Zweeler", "Tour1", "Tour2", "Tour3", "Classics1", "Point"]]
RIDERS_CATEGORIES = RIDERS_CATEGORIES.rename(columns = {"Point": "Classics2"})

RIDERS_CATEGORIES = pd.merge(RIDERS_CATEGORIES, RACE_POINTS_CLASSICS3, how = "left", left_on = "RiderName_Zweeler", right_on = "Navn")
RIDERS_CATEGORIES = RIDERS_CATEGORIES[["RiderName_Zweeler", "Tour1", "Tour2", "Tour3", "Classics1", "Classics2", "Point"]]
RIDERS_CATEGORIES = RIDERS_CATEGORIES.rename(columns = {"Point": "Classics3"})

RIDERS_CATEGORIES.Tour1 = RIDERS_CATEGORIES.Tour1.fillna(0)
RIDERS_CATEGORIES.Tour2 = RIDERS_CATEGORIES.Tour2.fillna(0)
RIDERS_CATEGORIES.Tour3 = RIDERS_CATEGORIES.Tour3.fillna(0)
RIDERS_CATEGORIES.Classics1 = RIDERS_CATEGORIES.Classics1.fillna(0)
RIDERS_CATEGORIES.Classics2 = RIDERS_CATEGORIES.Classics2.fillna(0)
RIDERS_CATEGORIES.Classics3 = RIDERS_CATEGORIES.Classics3.fillna(0)

RIDERS_CATEGORIES = RIDERS_CATEGORIES.astype({
    "Tour1": int,
    "Tour2": int,
    "Tour3": int,
    "Classics1": int,
    "Classics2": int,
    "Classics3": int
})

RIDERS_CATEGORIES["Total"] = RIDERS_CATEGORIES["Tour1"] + RIDERS_CATEGORIES["Tour2"] + RIDERS_CATEGORIES["Tour3"] + RIDERS_CATEGORIES["Classics1"] + RIDERS_CATEGORIES["Classics2"] + RIDERS_CATEGORIES["Classics3"]

def percentage_of_total(category, total):
    if total == 0:
        return round(0, 2)
    else:
        return round(category / total * 100, 1)

RIDERS_CATEGORIES["Tour1p"] = RIDERS_CATEGORIES.apply(lambda row: percentage_of_total(row["Tour1"], row["Total"]), axis = 1)
RIDERS_CATEGORIES["Tour2p"] = RIDERS_CATEGORIES.apply(lambda row: percentage_of_total(row["Tour2"], row["Total"]), axis = 1)
RIDERS_CATEGORIES["Tour3p"] = RIDERS_CATEGORIES.apply(lambda row: percentage_of_total(row["Tour3"], row["Total"]), axis = 1)
RIDERS_CATEGORIES["Classics1p"] = RIDERS_CATEGORIES.apply(lambda row: percentage_of_total(row["Classics1"], row["Total"]), axis = 1)
RIDERS_CATEGORIES["Classics2p"] = RIDERS_CATEGORIES.apply(lambda row: percentage_of_total(row["Classics2"], row["Total"]), axis = 1)
RIDERS_CATEGORIES["Classics3p"] = RIDERS_CATEGORIES.apply(lambda row: percentage_of_total(row["Classics3"], row["Total"]), axis = 1)

RIDERS_CATEGORIES = RIDERS_CATEGORIES[[
    "RiderName_Zweeler",
    "Total",
    "Tour1", 
    "Tour1p",
    "Tour2",
    "Tour2p",
    "Tour3",
    "Tour3p",
    "Classics1",
    "Classics1p",
    "Classics2",
    "Classics2p",
    "Classics3",
    "Classics3p"
]]

RIDERS_CATEGORIES = RIDERS_CATEGORIES.rename(columns = {
    "RiderName_Zweeler": "Navn",
    "Tour1": "Tour 1",
    "Tour1p": "%",
    "Tour2": "Tour 2",
    "Tour2p": "%",
    "Tour3": "Tour 3",
    "Tour3p": "%",
    "Classics1": "Classics 1",
    "Classics1p": "%",
    "Classics2": "Classics 2",
    "Classics2p": "%",
    "Classics3": "Classics 3",
    "Classics3p": "%"
})

,Navn,Total,Tour 1,%,Tour 2,%,Tour 3,%,Classics 1,%,Classics 2,%,Classics 3,%
0,"Aberasturi, Jon",0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1,"Abrahamsen, Jonas",0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2,"Ackermann, Pascal",0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
3,"Adrià, Roger",0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,"Aert, Wout van",0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917,"Zingle, Axel",0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
918,"Zoccarato, Samuele",0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
919,"Zubeldia, Unai",0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
920,"Zukowsky, Nick",0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [45]:
RIDER_OUTPUT_MAIN.to_html("../outputtables/RytteroversigtTabel.html", table_id = "filterabletable", index = False)
RIDER_OUTPUT_CHEAPO.to_html("../outputtables/RytteroversigtCheapoTabel.html", table_id = "filterabletable", index = False)
STANDINGS.to_html("../outputtables/StillingTabel.html", table_id = "filterabletable", index = False)
RACE_OUTPUT.to_html("../outputtables/LøbTabel.html", table_id = "filterabletable", index = False)
RACE_POINTS.to_html("../outputtables/LøbsresultaterTabel.html", table_id = "filterabletable", index = False)
RIDERS_CATEGORIES.to_html("../outputtables/KategoriKongerne.html", table_id = "filterabletable", index = False)